In [ ]:
import speech_recognition as sr
import PySimpleGUI as sg
import threading
from transformers import BartForConditionalGeneration, BartTokenizer

def recognize_speech(window):
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()
    transcribed_text = ""

    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        while listening:
            try:
                audio = recognizer.listen(source, timeout=5)  # Adjust timeout as needed
                text = recognizer.recognize_google(audio)
                window.write_event_value('-TEXT-', text + '\n')
                save_transcription(text + '\n')
                transcribed_text += text + '\n'
            except sr.UnknownValueError:
                pass
            except sr.RequestError as e:
                print("Error:", e)
    return transcribed_text

def bart_summarization(paragraph):
    model_name = 'facebook/bart-large-cnn'
    tokenizer = BartTokenizer.from_pretrained(model_name)
    model = BartForConditionalGeneration.from_pretrained(model_name)

    inputs = tokenizer.encode("summarize: " + paragraph, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

def start_listening(window):
    global listening
    listening = True
    threading.Thread(target=recognize_speech, args=(window,), daemon=True).start()
    window['-START-'].update(disabled=True)
    window['-STOP-'].update(disabled=False)

def stop_listening(window):
    global listening
    listening = False
    window['-START-'].update(disabled=False)
    window['-STOP-'].update(disabled=True)

    # Retrieve the already transcribed text from the window:
    transcribed_text = window['-OUTPUT-'].get()  # Get the text from the Output element
    summarized_text = bart_summarization(transcribed_text)
    window['-SUMMARY-'].update(summarized_text)

def save_transcription(text):
    with open(r'C:\Users\mitul\OneDrive\Desktop\transcription.txt', 'a') as file:  # Use 'a' for append mode
        file.write(text)

listening = False

layout = [
    [sg.Text("Live Speech to Text", font=("Arial", 16))],
    [sg.Output(size=(80, 20), key='-OUTPUT-', text_color='black')],
    [sg.Text("Summary:", font=("Arial", 12))],
    [sg.Text(size=(80, 5), key='-SUMMARY-', text_color='black')],
    [sg.Button('Start', key='-START-', size=(10, 1)), sg.Button('Stop', key='-STOP-', size=(10, 1), disabled=True)],
    [sg.Button('Save Transcript', key='-SAVE-', size=(15, 1))]
]

window = sg.Window('Live Speech to Text', layout)

while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED:
        break
    elif event == '-START-':
        start_listening(window)
    elif event == '-STOP-':
        stop_listening(window)
    elif event == '-TEXT-':
        window['-OUTPUT-'].print(values[event])
    elif event == '-SUMMARY-':
        window['-SUMMARY-'].update(value=values[event])  # Update the summary text
    elif event == '-SAVE-':
        with open(r'C:\Users\mitul\OneDrive\Desktop\transcription.txt', 'w') as file:
            file.write(window['-OUTPUT-'].get())  # Save the transcription

window.close()
